In [ ]:
#In this notebook we include an implementation of the different linear relaxations for maxcut that we consider.
#We use mosek as solver, with an academic licence.


<h1>Semidefinite and linear relaxations for the maximum cut problem<h1>

<h1>Table of contents <h1>   
    0. Load packages, functions <br>
    
    

<h2>  0. Load packages, functions<h2>

In [ ]:
#Packages 

using JuMP, LinearAlgebra, MosekTools


<h3>Functions<h3>

In [ ]:

#function that computes the total sum_{i<j}W_ij. 
#This quantity is used in various objective functions. 
#@param matrix adjMatrix: matrix for which to compute to total sum of the entries.
function Wtot(adjMatrix)
     dims = size(adjMatrix)[1]
     counter = 0
     for i = 1:dims
        for j = i:dims
            counter = counter+adjMatrix[i,j]
            end
        end
  return(counter)
end


In [ ]:
#function to compute the semidefinite relaxation for the maxcut problem
#@param matrix ajdMatrix: the adjancency matrix of the graph for which we wish to compute
# the semidefinite relaxation of max cut.
function GoemansPrimal(adjMatrix)
  
    dims = size(adjMatrix)[1]
    m = Model(Mosek.Optimizer)
    set_silent(m)    
    @variable(m, X[1:dims,1:dims], PSD)
    [@constraint(m, X[i,i]==1) for i in 1:dims]
   @objective(m,Max,tr(-adjMatrix*X))
    status = optimize!(m)
      return([value.(X),0.5*Wtot(adjMatrix)+0.25*objective_value(m), objective_value(m),solve_time(m)])
end


In [ ]:
#function to compute the linear programing inner approximation of the semidefinite relaxation
# for maxcut, as defined in program SD(S) in the paper. 
#This functions uses the eigenvectors of the adjacency matrix as the elements of S.
#@param matrix adjMatrix: the matrix for which we compute the relaxation.
function SDeigen(adjMatrix)
    dims = size(adjMatrix)[1]                                                 
    eigenFacto =  eigen(adjMatrix)
    eigenVals = eigenFacto.values
    eigenVectors = eigenFacto.vectors
   
    eigenVectsAmmount = length(eigenVectors[1,:])      

    m = Model(Mosek.Optimizer)
    set_silent(m)  
    @variable(m,lambda[1:eigenVectsAmmount]>=0)
  
     X= lambda[1]*eigenVectors[:,1]
    for i in 2: eigenVectsAmmount
        
        X= hcat(X, lambda[i]*eigenVectors[:,i])
           
    end
      
    Y = X*transpose(eigenVectors)
  
  #  Z = eigenVectors * Y
    [@constraint(m,Y[i,i]<=1) for i in 1:dims]
     @objective(m,Max,tr(-adjMatrix*Y))
      status = optimize!(m)     
    return([value.(Y),0.5*Wtot(adjMatrix) + (1/4)*objective_value(m),value.(lambda),eigenVectors,objective_value(m)])  
end



#function to compute the linear programing inner approximation of the semidefinite relaxation
# for maxcut, as defined in program SD(S) in the paper. 
#This functions uses the eigenvectors of the adjacency matrix as the elements of S and 
# a eigenbasis of an optimal solution found for program SP, computed by using the function eigenRelaxation.
#@param matrix adjMatrix: the matrix for which we compute the relaxation.
#@param matrix YsolEigen solution to the program SP, using the function eigenRelaxation.
function SDeigenEnhanced(adjMatrix,YsolEigen)
    dims = size(adjMatrix)[1]                                                 
      eigenVectors =  eigen(adjMatrix).vectors
    eigenVectorsEnhanced = hcat(eigenVectors,eigen(YsolEigen).vectors)
     eigenVectsEnhancedAmmount = length(eigenVectorsEnhanced[1,:])      

    m = Model(Mosek.Optimizer)
    set_silent(m)  
    @variable(m,lambda[1:eigenVectsEnhancedAmmount]>=0)
  
     X= lambda[1]*eigenVectorsEnhanced[:,1]
    for i in 2: eigenVectsEnhancedAmmount
        
        X= hcat(X, lambda[i]*eigenVectorsEnhanced[:,i])
           
    end
      
    Y = X*transpose(eigenVectorsEnhanced)
  
  #  Z = eigenVectors * Y
    [@constraint(m,Y[i,i]<=1) for i in 1:dims]
     @objective(m,Max,tr(-adjMatrix*Y))
      status = optimize!(m)     
    return([value.(Y),0.5*Wtot(adjMatrix) + (1/4)*objective_value(m),value.(lambda),eigenVectors,objective_value(m)])  
end




In [ ]:

#This function computes the dual of the eigenRelaxation, which corresponds to program DSP in the paper.
#@param matrix adjMatrix: the matrix for which we compute the dual of the SP relaxation.                                                                        
function EigenRelaxationDual(adjMatrix)
 eigenvectors = eigen(adjMatrix).vectors                                                           
 dims = size(adjMatrix)[1]
 m = Model(Mosek.Optimizer)
 set_silent(m)                                                                
 @variable(m, X[1:dims,1:dims], Symmetric)
 @variable(m,betas[1:dims]>=0)
  Z =  zeros(dims,dims)
  [Z = Z+ betas[i]*eigenvectors[:,i]*transpose(eigenvectors[:,i]) for i in 1:dims]
                                                            
  @variable(m,deltas[1:dims,1:dims],Symmetric)
  @constraint(m,deltas.>=0)                                                          
 [@constraint(m,deltas[i,i]==0) for i in 1:dims]   
                                                            
  @variable(m,xi[1:dims,1:dims],Symmetric)
 [@constraint(m,xi[i,i]==0) for i in 1:dims]       
  @constraint(m,xi.>=0)                                                          
   
   for i in 1:dims
        for j in i+1:dims
   @constraint(m,X[i,j]==0.5*(deltas[i,j]-xi[i,j]))
         end
      end
                                                                                                                                                                                      
 @constraint(m,X+Z .== adjMatrix)
  @objective(m,Min,-tr(X)+0.5*sum(deltas)+0.5*sum(xi))                                                     
   # print(m)
  status = optimize!(m)
  return([value.(X),0.5*Wtot(adjMatrix) + (1/4)* objective_value(m),value.(betas),objective_value(m),value.(deltas),value.(xi),value.(Z)])
                                                        
end                

In [ ]:
#function to compute the eigenRelaxation using the eigenvectors of the adjacency matrix, as
# defined in program SP in the paper. 
#@param matrix adjMatrix: the adjacency matrix of the graph for which we compute the linear relaxation.
#@param matrix parameters: constraints added to the program, which consits of matrices of the form vv^t where
# v is a eigen vector of the adjacency matrix, or a standard basis vector. We use the constraints written in this form
# as this results in a  faster parsing of the optmization program. 
#The parsing is handled by JUMP package, and it is passed to the solver Mosek. To find the parameters, we call the
#function giveEigenRelParameters.
function eigenRelaxation(adjMatrix,parameters)
    n = size(adjMatrix)[1]
    m = Model(Mosek.Optimizer)
    set_silent(m)
      @variable(m,-1<=x[1:n,1:n]<=1,Symmetric )
      Y = vec(x)
    
      @constraint(m,parameters[2]*Y .>=parameters[3])
     # @constraint(m, [i = 0:n-1],  x[1+i*(n+1)] == 1)
       @constraint(m, [i = 1:n],  x[i,i] == 1)
       @objective(m,Max,dot(-vec(transpose(adjMatrix)),Y))
      status = optimize!(m)
     return([value.(x),0.5*Wtot(adjMatrix) + (1/4)* objective_value(m),objective_value(m),solve_time(m)])
end      

#function to compute the parameters needed to compute the eigenRelaxation. i.e, the matrices vv^t where
# v is a eigenvector of the adjacency matrix, which are then flattened and returned in matrix form.
# We return the vectors in this form as this results in a 
# faster parsing of the optmization program. The parsing is handled by JUMP package, and it is passed
# to the solver Mosek.
#@param matrix adjMatrix: the matrix for which we compute the eigenvectors and return them in appropiate form.
#@param bool useStandardBasis: indicates if we should add to the eigenvectors the standard basis vectors e_i.
function giveEigenRelParameters(adjMatrix,useStandardBasis)
     objectiveVector =  -vec(transpose(adjMatrix))                                                      
     dim = size(adjMatrix)[1]
    vectos = eigen(adjMatrix).vectors
     vectos = round.(vectos,digits=7)
     if(useStandardBasis == true)
              vectos =  hcat(vectos,Matrix{Float64}(I, dim, dim))
       end
           numRestrictions = size(vectos)[2]
           b = zeros(numRestrictions)                                                     
     constraintMatrix =  createRestrictionMatrix(vectos)
     return([objectiveVector,constraintMatrix,b])                                                       
 end           

 
# function to create a matrix containing the restrictions added to the linear program eigenRelaxation. The
# restrictions are of the form vv^t. These matrices are flattened and returned in matrix form.
# @param vectos matrix of eigenvectors to create the constraints of the form vv^T.  
function createRestrictionMatrix(vectos)
    # since vv^T is symetric, we can ignore the transpose in vec(transpose())
    
     temporalMat= Matrix{Float64}(undef, size(vectos)[1], size(vectos)[1])
     restrictionMatrix = vec(vectos[:,1]*transpose(vectos[:,1]))
      for j in 2: size(vectos)[2]
         mul!(temporalMat, vectos[:,j],transpose(vectos[:,j]))
        restrictionMatrix = hcat(restrictionMatrix,vec(temporalMat))

    end
    return(transpose(restrictionMatrix))
end

 


In [ ]:

# function to create and return a fooling graph, as defined in the paper in section 4.1.4.
# @param int k: the regularity of the k-regular part of the graph.
# @param int l: the number of edges joining the two components.
# @param bool replaceJ: indicates is we should take replacement for the vertex in the regular component.
# default is false.
function foolingGraph(n,k,l,replaceJ=false)

    regularComp = Matrix(adjacency_matrix(random_regular_graph(n,k)))
    squareRoot = Int8(round(sqrt(n)))
    completeBipartiteComp = Matrix(adjacency_matrix(complete_bipartite_graph(squareRoot,squareRoot)))
    empty = zeros(2*squareRoot,n)
    matrix = [regularComp transpose(empty);empty completeBipartiteComp]
    ientries =  sample(1:2*squareRoot,l, replace = true)
    jentries =  sample(1:n,l, replace = replaceJ)
    for s in 1:l
        matrix[n+ientries[s],jentries[s]]=1
        matrix[jentries[s],n+ientries[s]]=1
    end
    return(matrix,Graph(matrix))
end


#function to invert the values contained in an array.
#param array B: array for which we compute the inverses. Then i-th entry
# of the returned array is 1/i where i is the i-th entry of B.

function invertArray(B)
    p=zeros(length(B))
    for i = 1:length(B)
        p[i] = 1/B[i]
    end
    return(p)
end

In [ ]:
#function to compute quotients of 0.25lambda_n*n vs z_gw and z_sp vs z_gw for a fooling graph.
# This function is used to compute the values in tables 4 and 5.
#@param int n: number of vertices of a fooling graph.
# @param int k: the regularity of the k-regular part of the graph.
# @param int l: the number of edges joining the two components.
function tableEigenFooling(n,k,l)
   
    Means = Float64[]
    standardDev = Float64[]
    results = zeros(6)
   
         GwZeigCurrentValue = zeros(5)
          GWEigenBound = zeros(5)
        for j= 1: 5
       
             G = foolingGraph(n,k,l)[1]
            GoemansSol= GoemansPrimal(G)[2]
            GwZeigCurrentValue[j] =eigenRelaxation(G, giveEigenRelParameters(G,true))[2]/ GoemansSol
            GWEigenBound[j] = (0.5*Wtot(G)-0.25*n* eigs(G,nev=1,which=:SR)[1][1] ) / GoemansSol
           
           
        end
            results[1] = mean(GwZeigCurrentValue)
         results[2] = mean(GWEigenBound)
        results[3] = std(GwZeigCurrentValue)
        results[4] = std(GWEigenBound)
        return(results)
end
